In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
# CPU will tak 50s infer
# !pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


In [ ]:
# Use GPU 3s infer
%%bash
CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [42]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [43]:
#load pdf files
loader = PyPDFDirectoryLoader("./pdf")
data = loader.load()

In [ ]:
print(data)

In [45]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [46]:
len(text_chunks)

28

In [ ]:
#get the third chunk
text_chunks[2]

In [48]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [49]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
# connect to google drive if need
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="./mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096,
    n_gpu_layers=30,
    n_threads=2,
    n_batch=521,
)

In [51]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [52]:
qa.run("What prevent thread not being execute")


llama_print_timings:        load time =   974.51 ms
llama_print_timings:      sample time =     2.40 ms /     8 runs   (    0.30 ms per token,  3336.11 tokens per second)
llama_print_timings: prompt eval time =  2290.60 ms /  1096 tokens (    2.09 ms per token,   478.48 tokens per second)
llama_print_timings:        eval time =   273.75 ms /     7 runs   (   39.11 ms per token,    25.57 tokens per second)
llama_print_timings:       total time =  2578.38 ms


' Use of Mutex in Rust'

In [ ]:
# Wait for input if need
# import sys

# while True:
#   user_input = input(f"Input Prompt: ")
#   if user_input == 'exit':
#     print('Exiting')
#     sys.exit()
#   if user_input == '':
#     continue
#   result = qa({'query': user_input})
#   print(f"Answer: {result['result']}")